# Timm and Barth 2011

This script provides a Python reference implementation of Timm and Barth 2011: *Accurate eye centre localisation by means of gradients* with some accompanying visualizations.

I use an artificial example to show how the algorithm works and visualize the theory behind it. Then I apply it to some pre-recorded eye images.

**TODO: List of possible changes/tests**
- Different eye-sizes (these are about 60-70 x 60-70 pixels).
- Lookup-table dynamically?
- Gaussian filter before and after as weights
- Different weights
- Effect of sigma, the free parameter
- **Export some graphs as pgf or eps or similar for thesis**

In [ ]:
%matplotlib inline
import itertools
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi
import scipy.misc

In [ ]:
num_eyes = 16
left_eyes = [plt.imread(f'eyes/eye_left_{i}.png') for i in range(num_eyes)]
right_eyes = [plt.imread(f'eyes/eye_right_{i}.png') for i in range(num_eyes)]

In [ ]:
def eyes_stat(eyes, fun, p=False):
    stat = fun(list(fun(eye) for eye in left_eyes))
    if p:
        print(fun, 'was', stat)
    return stat
min_left = eyes_stat(left_eyes, np.min, True)
max_left = eyes_stat(left_eyes, np.max, True)
min_right = eyes_stat(right_eyes, np.min, True)
max_right = eyes_stat(right_eyes, np.max, True)

def maprange(a, b, s):  # From rosetta code because why not copy it
    (a1, a2), (b1, b2) = a, b
    return  b1 + ((s - a1) * (b2 - b1) / (a2 - a1))

# ensure 0 ... 1
left_eyes = [maprange((min_left, max_left), (0, 1), eye) for eye in left_eyes]
right_eyes = [maprange((min_right, max_right), (0, 1), eye) for eye in right_eyes]

In [ ]:
def plot_eyes(eyes):
    plt.figure(figsize=(10,10))
    for i, eye in enumerate(eyes[:16], 1):
        plt.subplot(4, 4, i)
        plt.imshow(eye, cmap='gray')
plot_eyes(left_eyes)
plot_eyes(right_eyes)

In [ ]:
%matplotlib inline
SIGMA = 0.005 * 160  # 160 ~face width

# Precalculate displacement table
displace_center = np.array((100,100))
displace_table = np.indices((201,201)) - displace_center[:,None,None]
displace_table = displace_table[::-1,:,:] / (np.linalg.norm(displace_table[::-1,:,:], 2, 0, True) + 1e-10)
def lookup_displacement(ref, dimension):
    return displace_table[:,
      (displace_center[0] - ref[0]) : (displace_center[0] + dimension[0] - ref[0]),
      (displace_center[1] - ref[1]) : (displace_center[1] + dimension[1] - ref[1])]

def find_center(eye, show=False, expected=None):
    inp = eye.copy()
    
    gaussian = ndi.gaussian_filter(inp, SIGMA)
    
    gradient = np.stack((ndi.sobel(gaussian, 1), ndi.sobel(gaussian, 0)), 0)
    magnitude = np.sqrt(gradient[0] * gradient[0] + gradient[1] * gradient[1])
    
    g = gradient / (magnitude[None] + 1e-10)
    g[:, magnitude < (.8 * np.std(magnitude) + np.mean(magnitude))] = 0
    
    t = np.zeros(inp.shape)
    for c in itertools.product(range(inp.shape[0]), range(inp.shape[1])):
        c = np.array(c)
        d = lookup_displacement(c, inp.shape)
        t[c[0],c[1]] = np.mean(np.maximum(0, np.sum(d * g, 0)) ** 2)

    result = np.unravel_index(np.argmax(t), t.shape)
    
    # END OF ALGORITHM, FROM HERE ONLY PLOTTING
    if show:
        plt.figure(figsize=(20,15))
        plt.subplot(5, 2, 1)
        plt.imshow(inp, cmap='gray')
        plt.subplot(5, 2, 2)
        plt.imshow(gaussian, cmap='gray')
        plt.subplot(5, 2, 3)
        plt.imshow(gradient[0], cmap='gray')
        plt.subplot(5, 2, 4)
        plt.imshow(gradient[1], cmap='gray')
        plt.subplot(5, 2, 5)
        plt.imshow(g[0], cmap='gray')
        plt.subplot(5, 2, 6)
        plt.imshow(g[1], cmap='gray')
        plt.subplot(5, 2, 7)
        plt.imshow(t, cmap='jet')
        plt.subplot(5, 2, 8)
        plt.imshow(magnitude, cmap='jet')
        
        if expected is not None:
            plt.figure(figsize=(10,10))
            plt.xlim([0, inp.shape[0]])
            plt.ylim([inp.shape[1], 0])
            d = lookup_displacement(np.array(expected), inp.shape)
            plt.quiver(*d, color='gray', alpha=.3, angles='xy', scale_units='xy', scale=1)
            plt.quiver(*g, color='red', alpha=.8, angles='xy', scale_units='xy', scale=1)
        
        plt.figure(figsize=(10,10))
        plt.xlim([0, inp.shape[0]])
        plt.ylim([inp.shape[1], 0])
        d = lookup_displacement(result, inp.shape)
        plt.quiver(*d, color='gray', alpha=.3, angles='xy', scale_units='xy', scale=1)
        plt.quiver(*g, color='red', alpha=.8, angles='xy', scale_units='xy', scale=1)
    return result

In [ ]:
def mark_center(eye, center):
    inp = eye.copy()
    inp = np.tile(inp[...,None],(1,1,3))
    color = (.8, 0, .5)
    inp[center[0]-5:center[0]+5,center[1]] = color
    inp[center[0],center[1]-5:center[1]+5] = color
    return inp

for eye in (left_eyes + right_eyes)[:2]:
    center = find_center(eye, True)
    plt.figure()
    plt.imshow(mark_center(eye, center), cmap='gray')
None  # To avoid printing of axis object

In [ ]:
expected_center = np.array((30, 20))

f = plt.figure(figsize=(10,10))
a = f.gca()
plt.xlim([0, 80])
plt.ylim([80, 0])
a.set_aspect(1)
a.axis('off')

circle = plt.Circle(expected_center[::-1], 6, color='gray')
a.add_artist(circle)
f.canvas.draw()
width, height = (f.get_size_inches() * f.get_dpi()).astype(np.int)
image = np.fromstring(f.canvas.tostring_rgb(), dtype='uint8').reshape(height, width, 3).astype(np.float) / 255
image = image @ [.3,.59,.11]
image = ndi.zoom(image, .1)
scipy.misc.imsave('artifical_example_eye_center.png', image)

f = plt.figure(figsize=(4,4))
plt.gray()
a = f.gca()
plt.xlim([0, 80])
plt.ylim([80, 0])
a.set_aspect(1)

circle = plt.Circle(expected_center[::-1], 6, color='gray')
a.add_artist(circle)


plt.figure()
plt.imshow(image, cmap='gray', vmin=0,vmax=1)

center = find_center(image, True, expected=expected_center)
# Note that because the way saving etc. works here, that 29,23 seems to be more accurate
print(expected_center, 'is expected, got:', center)
plt.figure()
plt.imshow(mark_center(image, center), cmap='gray')
None

In [ ]:
a = np.arange(8).reshape(2,2,2)
b = np.arange(8).reshape(2,2,2)

r = np.zeros((2,2))
for i in np.indices((2,2)):
    r[i] = a[0,i] * b[0,i] + a[1,i] * b[1,i]
    
c = np.sum(a * b, 0)

print(np.all(a[0] * b[0] + a[1] * b[1] == r), np.all(c == r))

In [ ]:
marked_left = [mark_center(eye, find_center(eye)) for eye in left_eyes[:16]]
plot_eyes(marked_left)
marked_right = [mark_center(eye, find_center(eye)) for eye in right_eyes[:16]]
plot_eyes(marked_right)

In [ ]:
# plt.figure(figsize=(20,20))
# plt.quiver(*displace_table, angles='xy', scale_units='xy', scale=1)
def visualize_displacement(ref, dimension):
    displacement = lookup_displacement(ref, dimension)
    plt.figure(figsize=(20,20))
    plt.quiver(*displacement, angles='xy', scale_units='xy', scale=1)
    plt.ylim(plt.ylim()[::-1])
visualize_displacement(np.array((5,5)), np.array((50,50)))
visualize_displacement(np.array((25,25)), np.array((50,50)))